For a brief description of functional programming, the following is from [Wikipedia](https://en.wikipedia.org/wiki/Functional_programming):

In computer science, functional programming is a programming paradigm where programs are constructed by applying and composing functions. It is a declarative programming paradigm in which function definitions are trees of expressions that map values to other values, rather than a sequence of imperative statements which update the running state of the program.

In functional programming, **functions are treated as first-class citizens, meaning that they can be bound to names (including local identifiers), passed as arguments, and returned from other functions, just as any other data type can**. This allows programs to be written in a declarative and composable style, where small functions are combined in a modular manner.

Functional programming is sometimes treated as synonymous with **purely functional programming**, a subset of functional programming which treats all functions as deterministic mathematical functions, or **pure functions**. When a pure function is called with some given arguments, it will always return the same result, and cannot be affected by any mutable state or other side effects. This is in contrast with impure procedures, common in imperative programming, which can have side effects (such as modifying the program's state or taking input from a user). Proponents of purely functional programming claim that by restricting side effects, programs can have fewer bugs, be easier to debug and test, and be more suited to formal verification.

### Functions

- When a function defines a parameter with a default value, that parameter and all the parameters that follow are optional.

- The use of mutable objects as default values may lead to unintended behavior; see Mistake 1 [here](https://www.evernote.com/shard/s191/nl/21353936/06c8859d-90c6-4688-a5a9-ef37f71e38f2?title=The%2010%20Most%20Common%20Mistakes%20That%20Python%20Developers%20Make%20%7C%20Toptal). It is better to use None and add a check.

- Function arguments can also be supplied by explicitly naming each parameter and specifying a value. These are known as **keyword arguments**. 
    - With keyword arguments, the **order of the parameters doesn’t matter**.
    - Positional arguments and keyword arguments can appear in the same function call, provided that **all the positional arguments appear first**, values are provided for all nonoptional arguments, and no argument value is defined more than once. 
    
- A function can accept **a variable number of parameters** if an **asterisk (`*`)** is added to the last parameter name. If the last argument of a function definition begins with `**`, all the additional keyword arguments (those that don’t match any of the other parameter names) are placed in a dictionary and passed to the function. You can combine extra keyword arguments with variable-length argument lists, as long as the `**` parameter appears last. Indeed, this use of `*args` and `**kwargs` is commonly used to write wrappers and proxies for other functions.

### Parameter Passing and Return Values

- The underlying semantics of parameter passing doesn’t neatly fit into any single style, such as **“pass by value”** or **“pass by reference”**, that you might know about from other programming languages.
- Functions that mutate their input values or change the state of other parts of the program behind the scenes like this are said to **have side effects**. As a general rule, this is a programming style that is best avoided because such functions can become a source of subtle programming errors.
- The `return` statement returns a value from a function. If no value is specified or you omit the return statement, the None object is returned.

### Scoping Rules

- Variables in nested functions are bound using **lexical scoping** by **LEGB**: 
    - local: the current function's scope, 
    - enclosing: any enclosing scopes (like other containing functions, or closures),
    - global: the scope of the module that contains the code (also called the global scope) 
    - built-in. 

- To change the behaviors above, use `local` or `global`.
  
- Variables that are **only referenced** inside a function are implicitly global. To constrast, a variable that is **created or assigned** a value anywhere within the function’s body, it’s assumed to be a local even before the actual assignment since the interpreter sees everything (so if you refer to it you will suffer an `UnboundedError`),  unless explicitly declared as global. But if the variable is an **associative type**, and we are adding or changing part of it, the change seems to stay even if the function is exited. For examples, see Item 15 in [< Effective Python >](https://www.evernote.com/shard/s191/nl/21353936/80774c17-012b-c7d8-e425-a9b3eb0d55f3?title=Effective%20Python).

### Functions as Objects and Closures

- When the statements that make up a function are packaged together with the environment in which they execute, the resulting object is known as a **closure**. 
    - All functions have a `__globals__` attribute that points to the global namespace in which the function was defined. This always corresponds to the enclosing module in which a function was defined.
    - When nested functions are used, closures capture the entire environment needed for the inner function to execute, in the `__closure__` attribute.

- A closure can be a highly efficient way to preserve state across a series of function calls. Though Item 15 of [< Effective Python >](https://www.evernote.com/shard/s191/nl/21353936/80774c17-012b-c7d8-e425-a9b3eb0d55f3?title=Effective%20Python) seems to prefer using helper class, Page 100 of [< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References) cares more about efficiency.
    - Note that the data enclosed in closures are subject to the **late binding** behavior, which says that the values of variables used in closures are **looked up at the time the inner function is called, not when it is defined**. If the data is just assigned a value, there is no problem; however, if the data is generated, there can be counterintuitive behavior; see the 6th mistake in [this post](https://www.evernote.com/shard/s191/nl/21353936/06c8859d-90c6-4688-a5a9-ef37f71e38f2?title=The%2010%20Most%20Common%20Mistakes%20That%20Python%20Developers%20Make%20%7C%20Toptal).

In [3]:
'''
Functions are objects.
li = [foo, dir, help] # They can be in a list...
li[1]() # ...and hence can be called using an index (don't forget the parantheses!)
bar = foo # They can also have alias
actions = { 7: foo, 11: dir} # They can be put in a dict, which can fulfill the functionality of the switch statement.
'''

# Enclosing a function inside a fuction can be a way to factory functions
def foo(x): 
    mol = 422
    def bar(): # A function enclosed in a function is not available in outside scope...
        # Any data within the outside function scope (but not the global scope) 
        # and used in the inner function is enclosed in the closere (i.e. part of the closure)
        print(x, mol)
        print('bar')
    return bar # ... unless it is returned. Call foo will return a functions which will otherwise be GC'ed.

func1 = foo(42) # Note that func1 is *not* an alise of bar
func2 = foo(666)
# What actually happens...
# func1 and func2 now are associated to an object, respectively, while both these objects point to bar().
# The data (42 and 666) are stored in the aforementioned objects, which allows for functions to have different states.
# The relation of func1 and func2 to bar are like two instances of the a class.
# This mechanism of function associated with data is called closure.

# So the following will return False
print(func1 is func2)

# The closured data can be assessed by
print(func1.__closure__)  # Note that no parantheses for func1

# func1()
func2()

False
(<cell at 0x7f3ad4413760: int object at 0x7f3ad43ac310>, <cell at 0x7f3ad4413640: int object at 0x7f3ad8ea1e50>)
666 422
bar


### Decorators

- A **decorator** is a function whose primary purpose is to **wrap another function or class**. The primary purpose of this wrapping is to **transparently alter or enhance the behavior of the object being wrapped**.

    - Decorators are closures or wrappers of functions.
    - Once wrapped, the wrapped function replace the original function for any call to the original.

- When decorators are used, they must appear on their own line immediately prior to a function or class definition, with `@`. More than one decorator can also be applied.

- Definitely with decorators, performance is affected due to the overhang. Applications of decorators include **timing**, **logging** and **authetications**.

- A decorator can also **accept arguments**.

- Decorators can also be applied to class definitions. 
    - For class decorators, you should **always have the decorator function return a class object as a result**.

- Note that wrapping functions with a decorator can **erase the (possibly) valuable information about function name the docstring**. One can either explicitly propagate the function name and the docstring, or invoke the `functools.wraps` decorator; see [o-data-structures-algorithms-and-code-simplification](o-data-structures-algorithms-and-code-simplification.ipynb) for more descriptions.

In [ ]:
from functools import wraps

lower, upper = range(2)

def foo(x, y, acme=2):
    """
    x is an int, y is float, returns some stuff
    """
    print('foo')
    return 41

# Mechanism of decorators...
# Step 1: a wrapper of the original function - decorators (tracer in this case) can be put in a module for everyone to import.
def tracer(func):
    # Below is a parameterized decorator, that is, a decorator that takes an extra argument. See below on how it is achieved.
    @wraps(func)
    def ifunc(*args, **kwargs):
        # *args takes any number of positional arguments - produces a tuple in function definition (hence order is important).
        # **kwargs takes any number of keyword arguments - pruduces a dictionary. Keyword arguments should be behind positional arguments.
        print(args)
        print('calling', func.__name__)
        # Unpack the tuples and dictionaries - use of asterisks similar to dereference.
        rv = func(*args, **kwargs)
        print('exiting')
        return rv
    return ifunc
    # Also need to transport the meta data of function name and docstring before returning. 
    # But the following two lines have been fulfilled by @wraps(func)
    '''
    ifunc.__name__ = func.__name__
    ifunc.__doc__ = func.__doc__
    '''
    
    return ifunc

# Step 2: bind it back to the original one
foo = tracer(foo)

foo(41, 42, acme=90)

# The true syntax of decorators - by default decorator takes the function immediately below as the first parameter.
'''
@tracer
def foo():
    print 'foo'
'''
    
# Now let's go back and talk about parametrized decorator - it can be achieved by wrapping up decorators.
# Take tracer as an example which allows it to take the cases of lower and upper to return different messages.
'''
def tracer(case):
    def idec(func):
        def ifunc(*args, **kwargs):
            print args
            print 'calling' if case == lower else 'CALLING'
            rv = func(*args, **kwargs)
            print 'exiting'
            return rv
        return ifunc
    return idec
'''
    

### Generator and `yield`

- If a function uses the `yield` keyword, it defines an object known as a generator. A generator is a function that produces a sequence of values for use in iteration, i.e. some iterator object. 
    - The generator object executes the function whenever `__next__` is called. More specifically, When `__next__` is invoked (usually in a loop rather than the `__next__`), the generator function executes statements until it reaches a `yield` statement.
    - The yield statement produces a result at which point execution of the function stops until `__next__` is invoked again. Execution then resumes with the statement following yield. 
    - A generator function signals completion by returning or raising `StopIteration`, at which point iteration stops. It is never legal for a generator to return a value other than `None` upon completion. Generator objects also have a method `close()` that is used to signal a shutdown. When a generator is no longer used or deleted, `close()` is called. Normally it is not necessary to call `close()`, but you can also call it manually.

- Besides using `yield` in a function to make a generator, another way is to use __generator expression__, e.g. `gen = (x*x for x in range(5))`

- One application of generator is when we do not want the whole iteration to be read into memory. Another one concerns __YieldManager__ decorator (find out more about this).

- Generators are a special case of iterators; see [this post](https://stackoverflow.com/questions/2776829/difference-between-pythons-generators-and-iterators).

- Item 17 of [< Effective Python > ](https://www.evernote.com/shard/s191/nl/21353936/80774c17-012b-c7d8-e425-a9b3eb0d55f3?title=Effective%20Python) talks about suggestions of dealing with iterators/generators.
    - Beware of **functions that iterate over input arguments multiple times**. 
        - If these arguments are iterators, you may see strange behavior and missing values, since iterators are stateful; in particular, when it is exhausted, they stay exhausted. 
        - What is worse, functions that call iterators cannot tell the difference between an iterator that has no output and an iterator that had output and is now exhausted - you do not get any errors from iterating over an already exhausted iterator.
    - Python’s iterator protocol defines how **containers vs iterators** interact with the `iter` and `next` built-in functions, `for` loops, and related expressions.
    - You can easily **define your own iterable container type by implementing the `__iter__` method as a generator**.
    - You can **tell if an object is an iterator vs. a container** if calling `iter()` on it twice produces the same result. 
        - This is because the iterator protocol states that when an iterator is passed to the `iter()` built-in function, `iter` will return the iterator itself. 
        - In contrast, when a container type is passed to `iter()`, a new iterator object will be returned each time.

In [ ]:
# For example
def gen():
    yield 1
    yield 7
    yield 11
    yield 42
    
go = gen()
next(go) # or go.next(). Yields are similar to breakpoints in debug...

# ... until we hit the end and we get a StopIteration. So we can also do:
for item in gen():
    print item
    
# Another example: even number generator. (
# This is a function solution. To compare, a class solution must have __iter__ and next defined 
# (recall that the next method must raise StopIteration when appropriate.)
def gen(limit):
    val = 0
    while val < limit:
        yield val
        val += 2
        
# Another way to get a generator object besides yield - applying functions on a list

ps = [1,7,11,42]

def with_tax(p):
    return p * 1.2

a = map(with_tax, ps) # not efficient on large list
b = map(lambda p: p*1.2, ps) # lambda function
c = [ p * 1.2 for p in ps ] # list comprehension - still takes up a lot of memory by producing the whole list
d = [ p * 1.2 for p in ps if p > 10 ] # We can do filtering as well.
g = ( p * 1.2 for p in ps if p > 10 ) # It is not a tuple! We have created a generator object and we can calculate on the fly.

### Coroutines and `yield` Expressions

- Inside a function, the `yield` statement can also be used as an expression that appears on the right side of an assignment operator. A function that uses yield in this manner is known as a **coroutine**, and it executes in response to values being sent to it.

- Besides `yield` being on the right side rather than the left side, another important difference between generators and coroutines is an initial call to `__next()__` is necessary so that the coroutine executes statements leading to the first yield expression. At this point, the coroutine suspends, waiting for a value to be sent to it using the `send()` method of the associated generator object. The value passed to `send()` is returned by the `(yield)` expression in the coroutine. Upon receiving a value, a coroutine executes statements until the next yield statement is encountered. The requirement of **first calling `__next()__` on a coroutine is easily overlooked and a common source of errors**. Therefore, it is recommended that coroutines be wrapped with a decorator that automatically takes care of this step.

- Yet another difference of coroutine is a coroutine will typically **run indefinitely unless it is explicitly shut down or it exits on its own**. To close the stream of input values, use the `close()` method. Once closed, a `StopIteration` exception will be raised if further values are sent to a coroutine. The `close()` operation raises `GeneratorExit` inside the coroutine.

- (There are more in Python Essential References about coroutine that you have not read.)

### List comprehensions

- The general syntax for list comprehension is 
  
  `[expression for item1 in iterable1 if condition1
    for item2 in iterable2 if condition2
    ...
    for itemN in iterableN if conditionN ]`

- The `if` clause is optional; however, if it’s used, expression is evaluated and added to the result only if condition is true.

- The iteration variable in list comprehension is private. 
 
- [8 Levels of Using List Comprehensions](https://www.evernote.com/shard/s191/nl/21353936/4b03c738-f077-45f7-9014-98a698e4da83)

### The `lambda` Operator

- `lambda` opeators define autonymous functions. 

- The code defined with lambda must be a valid expression. Multiple statements and other non-expression statements, such as for and while, cannot appear in a lambda statement.

- `lambda` expressions follow the same scoping rules as functions.

### Recursion

- The function `sys.getrecursionlimit()` returns the current maximum recursion depth, and the function `sys.setrecursionlimit()` can be used to change the value. The default value is `1000`. Although it is possible to increase the value, programs are still limited by the stack size limits enforced by the host operating system. When the recursion depth is exceeded, a `RuntimeError` exception is raised.

- Care should be taken when using recursion in generators and couroutines, since the `yield` statement can actually cause the function to stop executing; see an example on Page 112 of [< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References).

- Care should also be taken when mixing recursive functions and decorators. If a decorator is applied to a recursive function, all inner recursive calls now get routed through the decorated version. 

### Function Attributes

- Functions can have arbitrary attributes attached to them. The primary use of function attributes is in highly specialized applications such as parser generators and application frameworks that would like to attach additional information to function objects.
- As with documentation strings, care should be given if mixing function attributes with decorators. If a function is wrapped by a decorator, access to the attributes will actually take place on the decorator function, not the original implementation. `functools.wraps` can help in this case as well.

### `eval()`, `exec()` and `compile()`

- The `eval(str [,globals [,locals]])` function executes an expression string and returns the result.

- The `exec(str [, globals [, locals]])` function executes a string containing arbitrary Python code.


## References
- [< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References), Chapter 1, 6.
- Bloomberg ENG Training 'Aspects of Functional Programming'.